# Do houses in university towns retain more or less value than houses in other regions during a recession?

## Background

This project is an elaboration on an assignment from the [Introduction to Data Science in Python](https://www.coursera.org/learn/python-data-analysis) class from University of Michigan with Coursera.

I will compare mean housing prices in university and non-university towns during a recession. I want to know whether houses in university towns retain more value than houses in non-university towns.

**Hypothesis**: Mean housing prices in university towns are less affected by recessions than mean housing prices in other non-university towns. I will calculate a statistic comparing mean price ratios in university and non-university towns to test this hypothesis.

Relevant terms:
* _quarter_ is a specific three month period; Q1 = Jan-Mar, Q2 = Apr-Jun, Q3 = Jul-Sep, Q4 = Oct-Dec
* _recession_ is a period of time; starts with 2 consecutive quarters of GDP decline, ends with 2 consequtive quarters of GDP growth
* _recession bottom_ is the quarter in recession with the lowest GDP
* _university town_ is a city with a high percentage of university students relative to total population
* _price ratio_ is defined as the mean housing price the quarter before a recession starts, divided by the mean housing price during the recession bottom

## Methods

I will evaluate market loss by comparing mean price ratios; i.e. the value of houses the quarter before a recession starts compared the the value of houses in the recession bottom.

I evaluate housing values in the 2008-2009 recession.

Housing prices are based on data from zillow. University towns are determined by a list on Wikipedia. Recessions are defined above and identified based on quarterly GDP from the Bureau of Economic Analysis.

Datasets:
* _Housing prices_ : median home sale prices by region in the United States; ```City_Zhvi_AllHomes.csv``` from [Zillow research data on all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv)
* _University towns_ : list of university towns in the United States; ```university_towns.txt``` from [Wikipedia page on university towns](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States)
* _GDP_ : GDP in annual and quarterly intervals in the United Statets; ```gdplev.xls``` from [US Dept. of Commerce, Bureau of Economic Analysis GDP over time](http://www.bea.gov/national/index.htm#gdp)

## Set up

In [1]:
# import libraries
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# set view options to display more rows and columns
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)

## Cleaning the data

To compare housing prices in university towns and non-university towns for the quarter before the recession and the recession bottom, we need a clean dataset. That dataset must list a few things.

Each record is a city. Columns are whether the town is a unviersity city or not, and its price ratio.

To get this information, I need to do a few things:
- Identify university towns.
- Identify recessions.
- Aggregate housing prices.

### Identify university towns

In [2]:
# function loads university town dataset and returns dataframe with two columns where 
# each row represents a university town and the two columns are...
# State: unabbreviated state name
# RegionName: name of neighborhood or city

def identify_university_towns():    
    
    # read in lines of text from university towns dataset
    with open('university_towns.txt') as uni_file:
        uni_text = uni_file.read().splitlines()
    
    # initialize lists of state and region names
    state_list = list()
    region_list = list()
    
    # iterate over lines of text
    for text in uni_text:
        
        # is this text a state name?
        # if text ends with "[edit]", it is a state name
        if (text[-6:] == "[edit]"):
            # save state name without [edit]
            state = text[:-6]
        
        # else this text is a region name
        else:
            
            # look for parenthetical statements in text
            i = text.find(" (")
            
            # if there are parenthetical statements
            if (i != -1):
                # save region name without parenthetical statements
                region = text[:i]
            
            # else there are no parenthetical statements
            else:
                # save region name as unaltered text
                region = text
                
            # append state and region to lists
            state_list.append(state)
            region_list.append(region)
            
    # create data frame from lists
    df = pd.DataFrame(list(zip(state_list, region_list)), columns = ["State", "RegionName"])
    
    # return data frame
    return df

### Identify recessions

In [3]:
# function loads GDP dataset 2000-2016 and returns dataframe with four columns
# where each row representsone recession, and the four columns are...
# Before: the quarter before a recession started
# Start: the quarter a recession started
# Bottom: the recession bottom, the quarter with the lowest GDP within a recession
# End: the quarter a recession ended

def identify_recession():
    # load GDP dataset in billions of chained 2009 dollars
    df = pd.read_excel('gdplev.xls', usecols = [4,6], names = ['Quarter', 'GDP'], skiprows = 219)
    # each record is the GDP in one quarter
    df.set_index('Quarter', inplace = True)
    # calculate GDP change between quarters
    df['GDP_change'] = df.diff()

    # initalize lists to track recession timing
    quarter_start_list = []
    quarter_before_start_list = []
    quarter_bottom_list = []
    quarter_end_list = []
    
    # initialize recession_bottom (will be reset at start of each recession)
    recession_bottom = 0

    # set recession flag to false
    in_recession = False

    # iterate through GDP quarter by quarter
    for i in np.arange(2, len(df)):

        # if we were not in a recession, but GDP decreased this quarter, 
        # and GDP decreased last quarter, then a recession started last quarter
        if ((not in_recession) & (df['GDP_change'].iloc[i] < 0) & (df['GDP_change'].iloc[i-1] < 0)):
            # set recession flag to true
            in_recession = True
            # record quarter when recession starts
            quarter_start_list.append(df.iloc[i-1].name)
            # record quarter before recession start
            quarter_before_start_list.append(df.iloc[i-2].name)
            # track quarter of recession bottom
            quarter_bottom = df.iloc[i].name
            # track recession bottom
            recession_bottom = df['GDP'].iloc[i]

        # if we are in a recession, and the current GDP is lower than the 
        # stored recession bottom, update the recesssion bottom
        elif (in_recession & (df['GDP_change'].iloc[i] < recession_bottom)):
            # update quarter of recession bottom
            quarter_bottom = df.iloc[i].name
            # update recession bottom
            recession_bottom = df['GDP_change'].iloc[i]

        # if we were in a recession, but GDP increased this quarter,
        # and GDP increased last quarter, then the recession ended this quarter
        elif (in_recession & (df['GDP_change'].iloc[i] > 0) & (df['GDP_change'].iloc[i-1] > 0)):
            # set recession flag to false
            in_recession = False
            # record quarter of recession bottom
            quarter_bottom_list.append(quarter_bottom)
            # record quarter when recession ends
            quarter_end_list.append(df.iloc[i].name)

    # if we're still in a recession at the end of the dataset
    if (in_recession):
        # store recession bottom and end as NA
        quarter_bottom_list.append(np.NaN)
        # record quarter when recession ends
        quarter_end_list.append(np.NaN)

    # combine lists in to a dataframe, where each column is a list
    df = pd.DataFrame([quarter_before_start_list, quarter_start_list,
                       quarter_bottom_list, quarter_end_list]).transpose()
    # name columns
    df.columns = ['Before', 'Start', 'Bottom', 'End']
    # return dataframe where each record is one recession
    return df

### Aggregate housing prices

In [4]:
# function loads housing price dataset for 2000-2016 and returns dataframe with 67 columns  
# where each row represents a region and the columns represent mean quarterly housing prices

def aggregate_housing_prices():
    # read in csv file
    df = pd.read_csv('City_Zhvi_AllHomes.csv')
    # drop columns before 2000
    df.drop(df.iloc[:, 3:51].columns, axis = 1, inplace = True)
    # drop region ID column
    df.drop(['RegionID'], axis = 1, inplace = True)
    # dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    # replace state abbreviations with state names
    df.replace({'State': states}, inplace = True)
    # set state and region name as multi-level index
    df.set_index(['State', 'RegionName'], inplace = True)
    # convert column names to date time
    df.columns = pd.to_datetime(df.columns)
    # average columns across quarters
    df = df.resample('Q', axis=1).mean()
    # convert column names to string
    df.columns = df.columns.to_period("Q").strftime("%Yq%q")
    # return dataframe
    return df

## Statistical Analysis

### Calculation

In [5]:
# function loads datasets above, runs a two-sample t-test to compare the mean
# price ratio of university and non-university towns during a recession.
# output will return three values in a tuple...
# 1) True or False: are the mean price ratios statistically significnatly different?
# the threshold for statistical significance was arbitrarily chosen as 0.01.
# 2) decimal: what's the p-value for t-test?
# a lower value indicates that it's less likely the means are the same.
# 3) university or non-university: which regions have a lower mean price ratio?
# a lower mean price ratio indicates a reduced market loss during a recession,
# the houses in these regions retained more value in a recession

def run_ttest():
    # load recession timing data
    recession_data = identify_recession()
    # find quarter before recession
    quarter_before = recession_data['Before'].iloc[0]
    # find quarter of recession bottom
    quarter_bottom = recession_data['Bottom'].iloc[0]

    # load housing data
    housing_data = aggregate_housing_prices()

    # calculate price ratio
    price_ratio = housing_data[quarter_before] / housing_data[quarter_bottom]
    price_ratio.name = 'PriceRatio'

    # load university town data
    uni_data = identify_university_towns()
    # create university column to describe whether a region is a university town
    # since this series only includes university towns, every record has a True value
    uni_data['University'] = True
    # set multi-index of state and region name
    uni_data.set_index(['State', 'RegionName'], inplace = True)

    # merge price ratio with university towns
    df = pd.merge(price_ratio, uni_data, how = 'left', left_index = True, right_index = True)
    # set any NaN university values as False
    df['University'].replace(np.nan, False, regex = True, inplace = True)

    university_price_ratio = df[df['University'] == True]['PriceRatio']
    non_university_price_ratio = df[df['University'] == False]['PriceRatio']

    test = ttest_ind(university_price_ratio, non_university_price_ratio, nan_policy = 'omit')

    # store p-value
    p = test.pvalue

    # interpret p-value
    # if p-value is less than 0.01, then our means are significantly different
    # but note that this interpretation is perhaps an overstatement of the meaning
    # of a p-value
    if (test.pvalue < 0.01):
        different = True
    else:
        different = False

    # interpret test statistic
    # if t statistic is negative, then the mean of university price ratios
    # is lower than the mean of non-university price ratios,
    # which represents a reduced market loss in recessions
    if (test.statistic < 0):
        better = "university town"
    else:
        better = "non-university town"

    # store interpretations and p-value in tuple
    result = (different, p, better)

    return result

run_ttest()

(True, 0.00433226366991648, 'university town')

### Interpretation

Statistically speaking, my analysis indicates that the mean price ratios are significantly different between university and non-university towns, and that houses in university towns have a reduced market loss relative to houses in non-university towns.

Generally speaking, my analysis indicates that houses in university towns retain more market value in a recession than non-university towns. This could indicate that investors should target university areas for less risky investments; if they had to sell on short-notice during a recession, they would lose less money in university areas.

That said, this is just one analysis, and there is a 1% chance this relationship was found by random chance and the mean housing prices are the same. Any investment decisions would warrant further research for general market trends and specific investment regions.